In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
df = sns.load_dataset("titanic")
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [2]:
for col in df.columns:
    if df[col].dtypes == bool:
        df[col] = df[col].astype(int)

In [5]:
list(df.select_dtypes(exclude="number").columns)

['sex', 'embarked', 'class', 'who', 'deck', 'embark_town', 'alive']

In [7]:
def grab_cols(df,cat_th=10,car_th=10):
    
    cat_cols = list(df.select_dtypes(exclude="number"))
    num_cols = list(df.select_dtypes(include="number"))
    
    num_but_cat = [col for col in num_cols if df[col].nunique()<cat_th]
    cat_but_car = [col for col in cat_cols if df[col].nunique()>car_th]
    
    cat_cols = cat_cols + num_but_cat
    cat_cols = [col for col in cat_cols if col not in cat_but_car]
    num_cols = [col for col in num_cols if col not in num_but_cat]
    
    print(f"Observations: {df.shape[0]}")
    print(f"Variables: {df.shape[1]}")
    print(f'cat_cols: {len(cat_cols)}')
    print(f'num_cols: {len(num_cols)}')
    print(f'cat_but_car: {len(cat_but_car)}')
    print(f'num_but_cat: {len(num_but_cat)}')
    
    return cat_cols,num_cols,cat_but_car
# help(grab_col_names)
cat_cols,num_cols,cat_but_car = grab_cols(df)

Observations: 891
Variables: 15
cat_cols: 13
num_cols: 2
cat_but_car: 0
num_but_cat: 6


In [10]:
df.isnull().sum()

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

In [ ]:
def cat_summary(df,col,plot=False):
    print(pd.DataFrame({col:df[col].value_counts(),
                        col+"_ratio":df[col].value_counts()/len(df)*100}))
    print("=====================================")
    
    if plot:
        sns.countplot(x=df[col],data=df)
        plt.show(block=True)

cat_summary(df,"survived",plot=True)

In [12]:
df.groupby("sex")["survived"].mean()

sex
female    0.742038
male      0.188908
Name: survived, dtype: float64

In [13]:
df.groupby("sex").agg({"survived":["mean"]})

,survived
,mean
sex,
female,0.742038
male,0.188908


In [19]:
def target_summary_with_cat(df,target,cat_col):
    print(pd.DataFrame({"target_mean":df.groupby(cat_col)[target].mean(),
                        "count":df.groupby(cat_col)[target].count()}
                       ))
    
target_summary_with_cat(df,"survived","sex")

        target_mean  count
sex                       
female     0.742038    314
male       0.188908    577


In [20]:
for col in cat_cols:
    target_summary_with_cat(df,"survived",col)

        target_mean  count
sex                       
female     0.742038    314
male       0.188908    577
          target_mean  count
embarked                    
C            0.553571    168
Q            0.389610     77
S            0.336957    644
        target_mean  count
class                     
First      0.629630    216
Second     0.472826    184
Third      0.242363    491
       target_mean  count
who                      
child     0.590361     83
man       0.163873    537
woman     0.756458    271
      target_mean  count
deck                    
A        0.466667     15
B        0.744681     47
C        0.593220     59
D        0.757576     33
E        0.750000     32
F        0.615385     13
G        0.500000      4
             target_mean  count
embark_town                    
Cherbourg       0.553571    168
Queenstown      0.389610     77
Southampton     0.336957    644
       target_mean  count
alive                    
no             0.0    549
yes            1.0 